In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# Import Feature Engineered Sales Transaction file
sales_df = pd.read_csv('Sales-Transactions-Modified.csv')

# Top selling Items 

In [5]:
# Find the no of units sold of each product
# Find the unit price of each product (max of price considered)
top_sell_items_df = sales_df.groupby('Product').agg({'Qty':'sum', 'Rate':'max'})

# Reset the index by converting the Product into a column
top_sell_items_df.reset_index(inplace=True)

# Rank the product by most Qty sold
top_sell_items_df['Top_Sell_Rank'] = top_sell_items_df['Qty'].rank(method='min',ascending=False).astype(int)
top_sell_items_df.head


<bound method NDFrame.head of                  Product   Qty    Rate  Top_Sell_Rank
0        1.25 COOLDRINKS     2   670.0            753
1    10" CLASSIFOAM-1200    31  1560.0            648
2      10" ESSFOAM LOOSE   310    35.0            435
3              10" GREEN     8  1610.0            722
4          10*10 CITIZEN   126   120.0            520
..                   ...   ...     ...            ...
780          ZEN-D CHEAP     2    70.0            753
781         ZEN-REALPACK    63    90.0            580
782             ZEND-1ST   520   100.0            370
783         ZEND-CLASSIC   706   120.0            335
784         ZEND-PREMIUM  1139   100.0            290

[785 rows x 4 columns]>

# Most popular items 

In [9]:
# Remove duplicate records at Product, Date and Party level
unique_order_items_df = sales_df.drop_duplicates(['Product','Date','Party'])

unique_order_items_df.head(20)

# Find the no of orders placed and the unique no of customers placed orders, of each product
most_popular_items_df = unique_order_items_df.groupby('Product').agg({'Date':'count', 'Party':'nunique'})
most_popular_items_df.columns=['No_of_Orders','No_of_Customers']

# Reset the index by converting the Product into a column
most_popular_items_df.reset_index(inplace=True)

most_popular_items_df.head(20)

,Product,No_of_Orders,No_of_Customers
0,1.25 COOLDRINKS,1,1
1,"10"" CLASSIFOAM-1200",21,8
2,"10"" ESSFOAM LOOSE",2,2
3,"10"" GREEN",1,1
4,10*10 CITIZEN,9,7
5,10*10 MORE,193,46
6,10*10 NILKAMAL,45,16
7,10*10 PAPER SHEET,20,13
8,10*10 PRIYA 2ND,1,1
9,10*10 RADHA KRISHNA,112,21


In [17]:
# Products with high no of orders can be considered as most frequently purchased items
# To find the most popular items, include the no of customers purchased and provide more weightage to products purchased by more customers

# Weighted No_of_Orders (W) = O * (C / M)
# O = No_of_Orders
# C = No_of_Customers purchased the product
# M = Maximum no of customers made transactions in the entire period

O = most_popular_items_df['No_of_Orders']
C = most_popular_items_df['No_of_Customers']
M = most_popular_items_df['No_of_Customers'].max()

most_popular_items_df['Weighted_No_of_Orders'] = O * (C / M)
most_popular_items_df.head(40)



,Product,No_of_Orders,No_of_Customers,Weighted_No_of_Orders
0,1.25 COOLDRINKS,1,1,0.004878
1,"10"" CLASSIFOAM-1200",21,8,0.819512
2,"10"" ESSFOAM LOOSE",2,2,0.019512
3,"10"" GREEN",1,1,0.004878
4,10*10 CITIZEN,9,7,0.307317
5,10*10 MORE,193,46,43.307317
6,10*10 NILKAMAL,45,16,3.512195
7,10*10 PAPER SHEET,20,13,1.268293
8,10*10 PRIYA 2ND,1,1,0.004878
9,10*10 RADHA KRISHNA,112,21,11.473171


In [18]:
# Rank the product by weighted no of orders
most_popular_items_df['Popularity_Rank'] = most_popular_items_df['Weighted_No_of_Orders'].rank(method='min',ascending=False).astype(int)

most_popular_items_df.head(20)

,Product,No_of_Orders,No_of_Customers,Weighted_No_of_Orders,Popularity_Rank
0,1.25 COOLDRINKS,1,1,0.004878,667
1,"10"" CLASSIFOAM-1200",21,8,0.819512,357
2,"10"" ESSFOAM LOOSE",2,2,0.019512,594
3,"10"" GREEN",1,1,0.004878,667
4,10*10 CITIZEN,9,7,0.307317,432
5,10*10 MORE,193,46,43.307317,81
6,10*10 NILKAMAL,45,16,3.512195,240
7,10*10 PAPER SHEET,20,13,1.268293,325
8,10*10 PRIYA 2ND,1,1,0.004878,667
9,10*10 RADHA KRISHNA,112,21,11.473171,157


In [21]:
most_popular_items_df.sort_values("Popularity_Rank", ascending=True)


,Product,No_of_Orders,No_of_Customers,Weighted_No_of_Orders,Popularity_Rank
77,14 GREEN,1464,205,1464.000000,1
748,TIRUMALA-50(16*20),1244,197,1195.453659,2
209,BLACK DOG-350ML,1268,181,1119.551220,3
763,VISHNU 250ML,1223,149,888.912195,4
764,VISHNU 300ML,1206,142,835.375610,5
...,...,...,...,...,...
440,OWN ITEM,1,1,0.004878,667
444,PAPER JUICE 150ML,1,1,0.004878,667
466,PEACOCK 13*16,1,1,0.004878,667
480,POLY 5*8 M.W,1,1,0.004878,667


# Merge rankings on product column 

In [24]:
# Merge Top Selling Items Rank and Popularity Rank dataframes
product_rankings_df = pd.merge(top_sell_items_df,most_popular_items_df,how='inner',on='Product')


# Get only the Product, Price and Rank columns
product_rankings_df = product_rankings_df[['Product','Rate','Top_Sell_Rank','Popularity_Rank']]
product_rankings_df.head(20)

,Product,Rate,Top_Sell_Rank,Popularity_Rank
0,1.25 COOLDRINKS,670.0,753,667
1,"10"" CLASSIFOAM-1200",1560.0,648,357
2,"10"" ESSFOAM LOOSE",35.0,435,594
3,"10"" GREEN",1610.0,722,667
4,10*10 CITIZEN,120.0,520,432
5,10*10 MORE,140.0,157,81
6,10*10 NILKAMAL,110.0,272,240
7,10*10 PAPER SHEET,70.0,441,325
8,10*10 PRIYA 2ND,100.0,658,667
9,10*10 RADHA KRISHNA,110.0,198,157


# Write product rankings in .csv file

In [25]:
product_rankings_df.to_csv('Product-Rankings.csv',index=False)

# Create a Pickle (.pkl) file with the Ranking dataframe

In [26]:
pickle.dump(product_rankings_df, open('prod_ranking_model.pkl','wb'))
